<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%203/NLP_08d_Clasificador_binario_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de texto con CNN
Implementemos un clasificador con Convolutional Neural Networks aplicado al análisis de sentimiento en Twitter usando la librería `Keras`.  
Aplicamos una primera capa de embeddings para convertir las palabras en vectores y luego entrenamos con una red CNN (seleccionando el max-pooling de cada filtro para obtener un vector por tweet).  
Para calcular los embeddings usamos:  
- Una capa de embeddings propia sobre los tweets
- Transfer Learning con los word embeddings de spaCy 
- Transfer Learning con los word embeddings de GloVe 

Implementado según el modelo planteado en [Convolutional Neural Networks for Sentence Classification](http://arxiv.org/abs/1408.5882)

In [1]:
import pandas as pd
import numpy as np
import re, string, spacy
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_colwidth = None
np.random.seed(123)

In [4]:
# Leemos los datos
#LOCAL:
#df = pd.read_csv('tweets_all.csv', index_col=None)
#GITHUB:
df = pd.read_csv('https://raw.githubusercontent.com/vicentcamison/idal_ia3/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%203/tweets_all.csv', index_col=None)

#seleccionamos columnas de interés
df = df[['content', 'polarity']]

#dejamos polaridades definidas
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

df.head()

,content,polarity
1,"@myendlesshazza a. que puto mal escribo\n\nb. me sigo surrando help \n\n3. ha quedado raro el ""cómetelo"" ahí JAJAJAJA",N
2,@estherct209 jajajaja la tuya y la d mucha gente seguro!! Pero yo no puedo sin mi melena me muero,N
3,Quiero mogollón a @AlbaBenito99 pero sobretodo por lo rápido que contesta a los wasaps,P
4,Vale he visto la tia bebiendose su regla y me hs dado muchs grima,N
5,@Yulian_Poe @guillermoterry1 Ah. mucho más por supuesto! solo que lo incluyo. Me habías entendido mal,P


In [ ]:
df.polarity.value_counts()

## Limpieza de texto
Usamos Spacy para separar el texto en tokens y mantener sólo las palabras importantes, dejando su lemma

In [ ]:
import re, string, spacy
nlp=spacy.load('es_core_news_md')

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, lemas=False):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Si lemas=True extraemos el lema, si no dejamos en minúsculas solamente.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lemma_.lower() if lemas else tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', tok) for tok in tokens] #no quitamos stop-words
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text
    

OSError: [E050] Can't find model 'es_core_news_md'. It doesn't seem to be a Python package or a valid path to a data directory.

## Preparamos el conjunto de datos
Convertimos el texto en *tokens* y asignamos una ID numérica a cada token.  
Convertimos a secuencias de longitud fija.  
La longitud de la secuencia viene dada por la longitud en tokens del tweet más largo. Sólo se conservan los tokens de las palabras en el vocabulario.

In [ ]:
#limpiamos texto y quitamos tweets que se han quedado vacíos
df.content=df.content.apply(clean_text, lemas=True)
df = df[df['content']!='']

#el conjunto de salida es la polaridad, hay que convertir a numérico para Keras
#codificamos 'P' como 1 y 'N' se queda como 0
Y=(df.polarity=='P').values*1

#Separamos entrenamiento y test
tweets_train, tweets_test, Y_train, Y_test = train_test_split(df.content,Y, test_size = 0.3, random_state = 42)

In [ ]:
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(tweets_train.values)
X_train = tokenizer.texts_to_sequences(tweets_train.values)
X_train = pad_sequences(X_train, padding='post')
word_index = tokenizer.word_index
print(f'Número de tokens distintos: {len(word_index)}')
MAX_SEQUENCE_LENGTH = X_train.shape[1]
max_features = len(word_index)+1
X_test = tokenizer.texts_to_sequences(tweets_test.values)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
X_train.shape

In [ ]:
X_train[0]

NameError: name 'X_train' is not defined

In [ ]:
tweets_train.values[0]

In [ ]:
word_index['después']

In [ ]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

## Word embeddins propios
Entrenamos una capa de embedding para aprender los WE con los textos de nuestro problema.

In [ ]:
#Creamos el modelo CNN en Keras
#Usamos como referencia el ejemplo de Keras: https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py
#pero quitamos la capa oculta intermedia para simplificar el modelo y dejarlo como en el artículo

#Parámetros de la red
embed_dim = 50
filters = 64
kernel_size = 3

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))

# añadimos una capa de convolución 1D que aprende
# filtros de grupos de palabras de tamaño kernel_size
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

# calculamos el max pooling:
model.add(GlobalMaxPooling1D())

# conectamos a una capa de salida de una unidad con activación sigmoide
model.add(Dense(1))
model.add(Activation('sigmoid'))

# compilamos el modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

### Pregunta:
¿de dónde vienen los tamaños de los parámetros de cada capa?

In [ ]:
#solución
print(max_features * embed_dim)
print(filters*embed_dim*kernel_size+filters)
print(filters+1)

In [ ]:
batch_size = 16
history = model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, verbose=2, validation_data=(X_test, Y_test))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('WE propios')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#los word embeddings aprendidos son los pesos de la primera capa
embeddings=model.get_weights()[0]

In [ ]:
embeddings.shape

In [ ]:
embeddings[1]

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
predict=model.predict(X_test, batch_size=1)
prediccion=(predict>0.5).tolist()

from sklearn.metrics import classification_report

print(classification_report(Y_test, prediccion, target_names=['N','P']))

### Ejercicio 1
Introduce una capa densa de 50 neuronas, con un Dropout con p=0.4 y una función de activación 'ReLU' entre la salida de la capa convolucional (después del MaxPooling) y la capa de salida. Compila y entrena con un tamaño de batch de 16 y 20 épocas.

In [ ]:
# Solución
#Creamos el modelo CNN en Keras
#Usamos como referencia el ejemplo de Keras: https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py
#pero quitamos la capa oculta intermedia para simplificar el modelo y dejarlo como en el artículo

#Parámetros de la red
embed_dim = 50
filters = 64
kernel_size = 3

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.2))

# añadimos una capa de convolución 1D que aprende
# filtros de grupos de palabras de tamaño kernel_size
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

# calculamos el max pooling:
model.add(GlobalMaxPooling1D())

model.add(Dense(50, activation='relu'))
model.add(Dropout(0.4))

# conectamos a una capa de salida de una unidad con activación sigmoide
model.add(Dense(1))
model.add(Activation('sigmoid'))

# compilamos el modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Compara los resultados con los anteriores

In [ ]:
batch_size = 16
history = model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, verbose=2, validation_data=(X_test, Y_test))

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('WE propios 2 capas')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
predict=model.predict(X_test, batch_size=1)
prediccion=(predict>0.5).tolist()

from sklearn.metrics import classification_report

print(classification_report(Y_test, prediccion, target_names=['N','P']))

## Word embeddings de spaCy
Aplicamos Transfer Learning usando los embeddings GloVe incluidos en el modelo de spaCy.

In [ ]:
nlp=spacy.load('es_core_news_md')
#Rellenamos los vectores con el valor en spaCy para nuestro vocabulario
EMBEDDING_DIM = nlp.vocab.vectors_length
embedding_matrix = np.zeros((max_features, EMBEDDING_DIM))
for word, i in word_index.items():
    if(i>max_features):
        break
    embedding_vector = nlp.vocab[word].vector
    if embedding_vector is not None:
        # las palabras que no están en spaCy serán cero.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
#Creamos el modelo CNN en Keras

#parámetros de la red
filters = 64
kernel_size = 3

embedding_layer = Embedding(max_features,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
batch_size = 16
history = model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, verbose=2, validation_data=(X_test, Y_test))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('TF con WE spaCy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
predict=model.predict(X_test, batch_size=1)
prediccion=(predict>0.5).tolist()

from sklearn.metrics import classification_report

print(classification_report(Y_test, prediccion, target_names=['N','P']))

### Word embeddings de FastText
Podemos usar cualquier conjunto de *word embeddings* con el formato `KeyedVectors` de Gensim para hacer Transfer Learning.  
WE descargados desde https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
from gensim.models import KeyedVectors
modelWE = KeyedVectors.load_word2vec_format('/Users/jovifran/Downloads/fasttext-sbwc.100k.vec')

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
#https://github.com/mquezada/starsconf2018-word-embeddings
modelWE = KeyedVectors.load_word2vec_format('~/Downloads/fasttext-sbwc.100k.vec')

EMBEDDING_DIM = modelWE.vector_size

embedding_matrix = np.zeros((max_features, EMBEDDING_DIM))
vectores = 0
for word, i in word_index.items():
    if(i<max_features):
        try:
            embedding_vector = modelWE[word]
        except:
            embedding_vector = np.zeros(EMBEDDING_DIM)
            # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vectores += 1
        
print("Cargados {} vectores en la matriz".format(vectores))

In [ ]:
max_features

In [ ]:
embedding_matrix.shape

In [ ]:
#Creamos el modelo CNN en Keras

#parámetros de la red
filters = 64
kernel_size = 3

embedding_layer = Embedding(max_features,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
batch_size = 16
history = model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, verbose=2, validation_data=(X_test, Y_test))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('TF con WE FastText')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
predict=model.predict(X_test, batch_size=1)
prediccion=(predict>0.5).tolist()

from sklearn.metrics import classification_report

print(classification_report(Y_test, prediccion, target_names=['N','P']))

### Inferencia en nuevos textos

Si queremos utilizar el clasificador con un texto nuevo hay que procesar el texto con la misma secuencia: limpieza, tokenizado y conversión en secuencia de la longitud adecuada:

In [ ]:
twt = 'estoy triste con el partido'
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences([clean_text(twt, lemas=True)]) #hay que pasar el texto a array
#padding the tweet to have exactly the same shape as `embedding_2` input

In [ ]:
twt = pad_sequences(twt, maxlen=X_train.shape[1], dtype='int32', padding='post', truncating='post', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)
if(np.round(sentiment) == 0):
    print("negativo")
elif (np.round(sentiment) == 1):
    print("positivo")

### Ejercicio 2
Repite el entrenamiento con la capa de embeddings de spaCy con `trainable = True` y compara los resultados.

In [ ]:
## Solución